In [6]:
import numpy as np
import tensorflow as tf
import sys
from sklearn.preprocessing import StandardScaler

#Load and Prepare the MNIST Dataset

## Vectorize function

In [7]:
def vectorize(data):
  return np.reshape(data, (1, -1))

## Normalize function

In [8]:
# Load the MNIST dataset and split it into training and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Vectorize the data
x_train = np.squeeze(np.array([vectorize(i) for i in x_train]))
x_test = np.squeeze(np.array([vectorize(i) for i in x_test]))

# Normalize the data
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Add a constant feature for bias
x_train = np.hstack((np.ones((len(x_train),1)), x_train))
x_test = np.hstack((np.ones((len(x_test),1)), x_test))

11490434/11490434 [==============================] - 0s 0us/step


#Multinonmial Logistic Regression Model

In [60]:
class MultinomialLogisticRegression():
  def __init__(self, lr=0.1, max_iter=1000, tol=0.01):
    self.lr = lr
    self.max_iter = max_iter
    self.tol = tol
    self.w = []

  def one_hot(self, y):
    num_classes = np.shape(np.unique(y))[0]
    y_one_hot = np.zeros((len(y), num_classes))
    for i in range(len(y)):
      y_one_hot[i, y[i]] = 1
    return y_one_hot

  def weight_init(self, num_features, num_classes):
    self.w = np.zeros((num_features, num_classes))

  def softmax(self, z):
    probs = np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1, 1)
    return probs

  def forward_pass(self, x):
    z = np.dot(x, self.w)
    y_hat_probs = self.softmax(z)
    return y_hat_probs

  def log_loss(self, y_one_hot, y_hat_probs):
    loss = -1 / (len(y_one_hot)) * np.sum(y_one_hot * np.log(y_hat_probs))
    return loss

  def accuracy(self, y, y_hat_probs):
    y_hat = np.argmax(y_hat_probs, axis=1)
    acc = 1.0 - np.count_nonzero(y_hat - y) / len(y)
    return acc

  def backward_pass(self, x, y_one_hot, y_hat_probs):
    dw = -1 / len(y_one_hot) * np.dot(x.T, (y_one_hot - y_hat_probs))
    return dw

  def fit(self, x, y):
    i = 0
    loss = 1
    y_one_hot = self.one_hot(y)
    self.weight_init(np.shape(x)[1], np.shape(y_one_hot)[1])
    while i < self.max_iter and loss > self.tol:
      y_hat_probs = self.forward_pass(x)
      loss = self.log_loss(y_one_hot, y_hat_probs)
      acc = self.accuracy(y, y_hat_probs)
      dw = self.backward_pass(x, y_one_hot, y_hat_probs)
      self.w -= self.lr * dw
      i += 1
      print('Iteration {}: loss = {:.4f}, accuracy = {:.4f}'.format(i, loss, acc))

  def predict(self, x):
    y_hat_probs = self.forward_pass(x)
    y_hat = np.argmax(y_hat_probs, axis=1)
    return y_hat

In [84]:
model = MultinomialLogisticRegression(lr=0.2, max_iter=500, tol=0.01)

In [85]:
model.fit(x_train, y_train)

Iteration 1: loss = 2.3026, accuracy = 0.0987
Iteration 2: loss = 1.1608, accuracy = 0.7361
Iteration 3: loss = 0.8864, accuracy = 0.7949
Iteration 4: loss = 0.7591, accuracy = 0.8230
Iteration 5: loss = 0.6819, accuracy = 0.8398
Iteration 6: loss = 0.6293, accuracy = 0.8501
Iteration 7: loss = 0.5908, accuracy = 0.8576
Iteration 8: loss = 0.5612, accuracy = 0.8632
Iteration 9: loss = 0.5376, accuracy = 0.8672
Iteration 10: loss = 0.5182, accuracy = 0.8709
Iteration 11: loss = 0.5019, accuracy = 0.8739
Iteration 12: loss = 0.4881, accuracy = 0.8761
Iteration 13: loss = 0.4761, accuracy = 0.8783
Iteration 14: loss = 0.4656, accuracy = 0.8803
Iteration 15: loss = 0.4563, accuracy = 0.8821
Iteration 16: loss = 0.4480, accuracy = 0.8836
Iteration 17: loss = 0.4405, accuracy = 0.8849
Iteration 18: loss = 0.4338, accuracy = 0.8864
Iteration 19: loss = 0.4276, accuracy = 0.8877
Iteration 20: loss = 0.4219, accuracy = 0.8889
Iteration 21: loss = 0.4168, accuracy = 0.8903
Iteration 22: loss = 0

In [86]:
y_hat = model.predict(x_test)

In [87]:
print('The empirical risk when using the missclassification loss function =', np.count_nonzero(y_hat - y_test) / len(y_test))
print('The empirical risk when using the squared error loss function =', sum((y_hat - y_test)**2) / len(y_test))

The empirical risk when using the missclassification loss function = 0.0763
The empirical risk when using the squared error loss function = 1.3825
